In [ ]:
import warnings

from jedi.api import file_name

warnings.filterwarnings("ignore", message="Protobuf gencode version")
from app.core.VDB.weaviateVDB import VectorDB
from app.core.Preprocessor.preprocessor import DocumentProcessor

path = './app/core/pdfs/전자금융감독규정(금융위원회고시)(제2025-4호)(20250205).pdf'
vdb = VectorDB()
vdb.reset()
processor = DocumentProcessor()
chunks = processor.preprocess(file_path = path)

In [ ]:
chunks

In [ ]:
from weaviate.classes.config import Property, DataType

properties=[
    Property(name="text", data_type=DataType.TEXT),
    Property(name="n_char", data_type=DataType.INT),
    Property(name="n_word", data_type=DataType.INT),
    Property(name="i_page", data_type=DataType.INT),
    Property(name="i_chunk_on_page", data_type=DataType.INT),
    Property(name="n_chunk_of_page", data_type=DataType.INT),
    Property(name="i_chunk_on_doc", data_type=DataType.INT),
    Property(name="n_chunk_of_doc", data_type=DataType.INT),
    Property(name="n_page", data_type=DataType.INT),
]

vdb.create_collection(name = 'LegalDB', properties=properties)

In [ ]:
vdb.show_collection()
vdb.set_collection(name = 'LegalDB')

In [ ]:
vdb.show_properties(name = 'LegalDB')

In [ ]:
objects = [{'text' : chunk.text,
            'n_char' : chunk.n_char,
            'n_word' : chunk.n_word,
            'i_page' : chunk.i_page,
            'i_chunk_on_page' : chunk.i_chunk_on_page,
            'n_chunk_of_page' : chunk.n_chunk_of_page,
            'i_chunk_on_doc' : chunk.i_chunk_on_doc,
            'n_chunk_of_doc' : chunk.n_chunk_of_doc,
            'n_page' : chunk.n_page} for chunk in chunks]
objects

In [ ]:
vdb.add_objects(objects = objects)

In [ ]:
query = '정보보호 총책임자는 어떻게 선정해야 하나요?'
result1 = vdb.query_bm25(query = query)
result2 = vdb.query_dense(query = query)
result3 = vdb.query_hybrid(query = query)

In [ ]:
result3

In [ ]:
retrieved_documents = [i['text'] for i in result3]
retrieved_documents

In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

# 클라이언트 초기화
client = OpenAI(
    api_key=os.environ.get("RUNPOD_API_KEY"),
    base_url=os.environ.get("RUNPOD_BASE_URL"),
)

# API 요청 보내기
try:
    response = client.chat.completions.create(
        model="K-intelligence/Midm-2.0-Base-Instruct",
        messages=[
            {
                "role": "user",
                "content": "서울의 날씨에 대해 알려줘."
            },
            {
                "role" : "system",
                "content" : "사용자의 질문을 바탕으로 사용할 도구를 선택하세요. 다른 말은 하지 않고 반드시 도구만 선택하세요."
            }
        ],
        temperature=0.7,
        tools=tools,
        tool_choice='auto'
    )

    # 응답에 tool_calls가 있는지 확인하여 분기 처리합니다.
    response_message = response.choices[0].message
    if response_message.tool_calls:
        tool_calls = response_message.tool_calls
        # 함수 호출 정보를 출력 (예시)
        print("모델이 도구 사용을 요청했습니다:")
        for tool_call in tool_calls:
            print(f"- 함수 이름: {tool_call.function.name}")
            print(f"- 인자: {json.loads(tool_call.function.arguments)}")
    else:
        # 일반 텍스트 응답인 경우 content를 출력합니다.
        print(response_message.content)

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

In [ ]:
from backends.app.core.financial_searchengine.financial_statements_extractor import financial_statements_extractor

extractor = financial_statements_extractor()

In [ ]:
result = extractor.extract_statement(corp_code = '005380')
result

In [ ]:
extractor.reports

In [ ]:
extractor.idx

In [ ]:
# system_prompt =  """당신은 재무제표 인덱스 검색가입니다. 주어진 목차를 바탕으로 재무제표가 몇번에 해당하는지 숫자를 말하세요. 
#         [주의사항]
#         1. 연결 재무제표, 연결 재무제표 주석, 재무제표 주석 등이 아닌 '재무제표' 라는 이름을 갖는 인덱스만 찾아야 합니다.
#         2. 다른 말은 절대 하지 말고 오로지 숫자만 말하세요. 예시:20
#         3. 인덱스를 찾을 수 없을 경우, '-1'만 출력하세요."""

# input = str({i : value for i, value in enumerate(extractor.dart.sub_docs('20250814003156')['title'].values.tolist())})
# from llm.llm import Midm
# llm = Midm()
# result = llm.call(system_prompt=system_prompt, user_input=input)

In [ ]:
# 현대자동차 : 005380
corp_code = '005930'

extractor.dart.company(corp_code)
# recent_rcept_no = extractor.get_recent_report(corp_code = corp_code)
# statement_idx = extractor.infer_statement_idx(rcept_no = recent_rcept_no)
# statement_url = extractor.get_statemnets_url(idx = statement_idx, rcept_no = recent_rcept_no)
# financial_statement = extractor._extract_statements(url = statement_url)


In [ ]:
extractor.dart.sub_docs('20250814003156').to_dict()

In [ ]:
[보고서 생성]
1. 기업에 대한 정보 (회사명, 사장이름, 홈페이지, 전화번호, 주로 하는 사업 등...)
2. 기업의 재무 상황
3. 웹 검색 기반 기업의 행보, 기업이 주로 진행중인 프로젝트 ? 사업 ? 


f"""

# {company_name} 기업에 대한 보고서

## 기업 정보
{company_info}
-
-
-

## 재무 상황
{financial_statement}


## 최신 기업 상황
{web_search_results}
-
-
-
## 결론 

"""

[질의응답 RAG]

-법률 도메인 질의응답 RAG

In [ ]:
## 1. ~~
{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '전영현',
 'corp_cls': 'Y',

 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.samsung.com/sec',
 'ir_url': '',
 'phn_no': '02-2255-0114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

## 2. 재무 상황

In [ ]:
from backends.app.services.rag_service import RagService
service = RagService()

In [ ]:


from app.services.report_service import ReportService
report_service = ReportService()
result = await report_service.generate_report('00266961')

In [2]:
from app.services.report_service import ReportService

service = ReportService()


In [2]:
result = await service.generate_report('00126380')


[DEBUG] Inside get_company_info
[DEBUG] Attempting to get corp_list for corp_code: 00126380
[DEBUG] Company found: 삼성전자. Fetching profile...
[DEBUG] Profile fetched for 삼성전자(주).
[DEBUG] 뉴스 검색 쿼리: 삼성전자(주) 최신 뉴스
[DEBUG] 수집된 뉴스 개수: 10
[DEBUG] 뉴스 1: 삼성전자, '북미 특화' 실외기·에어컨으로 글로벌 진출 속도 | 날짜: 
[DEBUG] 뉴스 2: 삼성전자, 작년 4분기 영업익 20조원...국내 최초 | 날짜: 
[DEBUG] 뉴스 3: 삼성전자, 4Q 잠정실적 이번 주 공개…영업익 20조 '신화' 쓴다 | 날짜: 
[DEBUG] 카테고리 분류 시작 - 뉴스 개수: 10
[DEBUG] 카테고리 분류 완료 - 관련뉴스: 9, 무관뉴스: 1
[DEBUG] 관련 뉴스 필터링 후: 9개
[DEBUG] 제목 유사도 필터링 후: 9개 (제거된 개수: 0)
[DEBUG] 카테고리별 기사 분포:
[DEBUG]   시장 및 경쟁: 1개
[DEBUG]   R&D 및 기술: 3개
[DEBUG]   생산 및 투자: 3개
[DEBUG]   실적 및 재무: 2개
[DEBUG] 선택된 카테고리: ['시장 및 경쟁', 'R&D 및 기술', '생산 및 투자', '실적 및 재무']
[DEBUG] 카테고리 '시장 및 경쟁'에서 선택: 삼성전자, '북미 특화' 실외기·에어컨으로 글로벌 진출 속도...
[DEBUG] 카테고리 'R&D 및 기술'에서 선택: 삼성전자, 인도서 최신 'AI 홈' 경험 선보여...
[DEBUG] 카테고리 '생산 및 투자'에서 선택: 삼성전자, 화성시 H-테크노밸리에 첨단 물류거점 조성...
[DEBUG] 카테고리 '실적 및 재무'에서 선택: 삼성전자, 작년 4분기 영업익 20조원...국내 최초...
[DEBUG] 최종 선택된 뉴스 개수: 4


In [3]:
corp_code = '00126380'
stock_code = '005930'
stock_name = 'NAVER'


await service.workflow._generate_stock_chart(corp_code, stock_code, stock_name)

'<div> <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n<script charset="utf-8" crossorigin="anonymous" integrity="sha256-4rD3fugVb/nVJYUv5Ky3v+fYXoouHaBSP20WIJuEiWg=" src="https://cdn.plot.ly/plotly-3.3.1.min.js"></script> <div class="plotly-graph-div" id="f59eb329-4c18-4529-aeb7-7b62d2219dfd" style="height:100%; width:100%;"></div> <script type="text/javascript">                window.PLOTLYENV=window.PLOTLYENV || {};                                if (document.getElementById("f59eb329-4c18-4529-aeb7-7b62d2219dfd")) {                    Plotly.newPlot(                        "f59eb329-4c18-4529-aeb7-7b62d2219dfd",                        [{"customdata":[52900,54000,53700,55600,55700,55800,55800,56000,56000,56900,58700,58400,58200,57300,57200,56600,56300,54500,54500,54000,54300,53700,53700,53600,54900,54700,54700,57600,57600,58500,60200,61700,60500,59800,61400,61800,60200,57800,58800,58800,57600,56100,53200,53500,53000,56400,55200,56200,56600,5